<a href="https://colab.research.google.com/github/Matya-Stavnitzky/Matya-Stavnitzky/blob/main/Cat_Mouse_Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Statements

In [1]:
!pip install tf-agents[reverb]
!pip install tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 25.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 39.2 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697630 sha256=044ccfca3561dd2b826ce6d31abd17aa56e8891a4ab5ac280be812d27a8ef792
  Stored in directory: /root/.cache/pip/wheels/3d/6f/b4/3991d4fae11d0ecb0754c11cc1b4e7745012850da4efaaf0b1
Successfully built gym
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensi

In [2]:
import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np
import random

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

#WORKING ENVIRONMENT

In [4]:
class CardGameEnv(py_environment.PyEnvironment):

  number_steps_index = 0
  mouse_index = 1
  cat_index = 2

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(2,), dtype=np.int32, minimum=[0,0], maximum=[1,1], name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,3), dtype=np.float32, minimum=0, name='observation')

    self._state = [0, 2, 4]
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = [0, 2, 4]
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.float32))


  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # check action
    #print('Action: ', action)
    #print('State before: ', self._state)
    cost_tally = [0, 0]
    if action[0] == 0: #if mouse
      if action[1] == 1: #if close
        cost_tally[0] -= 2
      else:
        #move the mouse
        mouse_location = self._state[self.mouse_index]
        if mouse_location == 1:
            self._state[self.mouse_index] = 3
        elif mouse_location == 2:
            self._state[self.mouse_index] = 1
        else:
            self._state[self.mouse_index] = 2

        cost_tally[0] += 1
        if self._state[self.mouse_index] == 3: #if mouse in room 3
          cost_tally[1] += 1
    else:
      if action[1] == 1: #if close
        cost_tally[1] -= 2
      else:
        #move the cat
        cat_location = self._state[self.cat_index]
        #print(f"cat starting location {cat_location}")
        if cat_location == 3:
            self._state[self.cat_index] = 4
        elif cat_location == 4:
            self._state[self.cat_index] = 5
        else:
            self._state[self.cat_index] = 3
        #print(f"cat ending location {self._state[self.cat_index]}")

        cost_tally[1] += 1
        if self._state[self.cat_index] == 3: #if cat in room 3
          cost_tally[0] += 1
    #print('Action data type: ', type(action))


    # calc state
    if self._state[self.mouse_index] == self._state[self.cat_index]:
      #print(f"cat and mouse met, state {self._state}")
      self._episode_ended = True
    self._state[0] += 1
    #print('State after: ', self._state)


    if self._episode_ended:
      reward = -10
      return ts.termination(np.array([self._state],  dtype=np.float32), reward)
    elif self._state[0] > 100:
      return ts.termination(np.array([self._state],  dtype=np.float32), 0)
    else:
      reward = cost_tally[action[0]] + np.random.normal(0, 0.1) #ADD RANDOM NOISE TO REWARD AS SPECIFIED BY PAPER
      return ts.transition(
          np.array([self._state], dtype=np.float32), reward=reward, discount=1.0)

In [5]:
environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=5)

/usr/local/lib/python3.10/dist-packages/tf_agents/specs/array_spec.py:352: RuntimeWarning: invalid value encountered in cast
  self._minimum[self._minimum == -np.inf] = low
/usr/local/lib/python3.10/dist-packages/tf_agents/specs/array_spec.py:353: RuntimeWarning: invalid value encountered in cast
  self._minimum[self._minimum == np.inf] = high


#Supervisors (AGENTS)

In [6]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  random_int = random.uniform(0,1)
  if random_int > epsilon:
    action = np.argmax(Qtable[state])
  else:
    action = random.choice([0, 1])
  return action

In [7]:
#epsilon_greedy_policy test


#helper functions

In [8]:
"""
33
34
35
-13
-14
-15
"""
def state_conversion_cat(state):
  if state[1] == 3:
    return int(state[2]-3)
  else:
    return int(state[2])

In [9]:
#test for state_conversion_cat
print(state_conversion_cat([0, 3, 3]))
print(state_conversion_cat([0, 3, 4]))
print(state_conversion_cat([0, 3, 5]))
print(state_conversion_cat([0, 1, 3]))
print(state_conversion_cat([0, 1, 4]))
print(state_conversion_cat([0, 1, 5]))

0
1
2
3
4
5


In [10]:
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [11]:
#test for inirialize_q_table
print(initialize_q_table(2, 3))

[[0. 0. 0.]
 [0. 0. 0.]]


In [12]:
"""
13
23
33
1-1
2-1
3-1
"""
def state_conversion_mouse(state):
  if state[2] == 3:
    return int(state[1]-1)
  else:
    return int(state[1]+2)

In [13]:
#test for start_conversion_mouse
print(state_conversion_mouse([0, 1, 3]))
print(state_conversion_mouse([0, 2, 3]))
print(state_conversion_mouse([0, 3, 3]))
print(state_conversion_mouse([0, 1, 4]))
print(state_conversion_mouse([0, 2, 4]))
print(state_conversion_mouse([0, 3, 5]))

0
1
2
3
4
5


In [14]:
def update_table(Qtable, state, action, learning_rate, reward, gamma, new_state):
  Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

In [39]:
#test for update_table
mouse_q = initialize_q_table(6, 2)
cat_q = initialize_q_table(6, 2)
#start state [0, 2, 4]
start_state_m = state_conversion_mouse([0, 2, 4])
#mouse move action 0
#next state [1, 3, 4]
end_state_m = state_conversion_mouse([1, 3, 4])
#updating table
update_table(mouse_q, start_state_m, 0, 0.1, -10, 0.9, end_state_m)
print(mouse_q)


[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [-1.  0.]
 [ 0.  0.]]


#Training Loop

In [16]:
def train(n_training_episodes, epsilon, alpha, gamma, env, max_steps, mouse_S, cat_S):
  for episode in range(n_training_episodes):

    # Reset the environment
    info, reward, done, state = env.reset()
    state = state[0]
    cat_state = state_conversion_cat(state)
    mouse_state = state_conversion_mouse(state)
    #step = 0
    done = False

    # repeat
    for step in range(max_steps):

      #decide if moving cat or mouse 0 = mouse
      move = random.choice([0, 1])
      if move == 0:
        action = epsilon_greedy_policy(mouse_S, mouse_state, epsilon)
      else:
        action = epsilon_greedy_policy(cat_S, cat_state, epsilon)


      info, reward, done, new_state = env.step([move, action])
      new_state = new_state[0] #CHECK THIS
      new_cat_state = state_conversion_cat(new_state)
      new_mouse_state = state_conversion_mouse(new_state)

      #print(f"starting state {state}, new_state {new_state}")
      #print(f"move made: {move} {action}")


      #update policy (need to go through equations to make sure it's accurate)
      if reward == -10: #need to check this NEGATIVE IMPACT FOR BOTH
          update_table(mouse_S, mouse_state, action, alpha, reward, gamma, new_mouse_state)  #update mouse
          update_table(cat_S, cat_state, action, alpha, reward, gamma, new_cat_state)  #update cat
      else:
        if move == 0:
          update_table(mouse_S, mouse_state, action, alpha, reward, gamma, new_mouse_state)  #update mouse
          if action == 0 and state[1] == 3: #are you supposed to update if reward == 0 IS THERE NOISE HERE
            update_table(cat_S, cat_state, action, alpha, 1, gamma, new_cat_state)
          else:
            update_table(cat_S, cat_state, action, alpha, 0, gamma, new_cat_state)
        else:
          update_table(cat_S, cat_state, action, alpha, reward, gamma, new_cat_state)  #update cat
          if action == 0 and state[2] == 3:
            update_table(mouse_S, mouse_state, action, alpha, 1, gamma, new_mouse_state)
          else:
            update_table(mouse_S, mouse_state, action, alpha, 0, gamma, new_mouse_state)


      # If done, finish the episode
      if not done:
        #print(f"finished episode, end state {new_state}")
        break

      # Our state is the new state
      state = new_state
      mouse_state = new_mouse_state
      cat_state = new_cat_state

  return [mouse_S, cat_S]

In [17]:
#Hyperparameters
alpha = 0.1
gamma = 0.9
epsilon = 0.1
n_training_episodes = 10000
learning_rate = 0.1  #alpha

# Evaluation parameters
n_eval_episodes = 100

#environment
env = CardGameEnv()

#Q-tables
state_space = 6
action_space = 2
mouse_Qtable = initialize_q_table(state_space, action_space)
cat_Qtable = initialize_q_table(state_space, action_space)

In [18]:
train(n_training_episodes, epsilon , alpha, gamma, env, 20,  mouse_Qtable, cat_Qtable)

[array([[-7.91939728, -7.71958462],
        [-3.03585736, -4.20119749],
        [ 0.        ,  0.        ],
        [-4.46299243, -5.3976775 ],
        [-2.50724767, -2.6687002 ],
        [-3.74303345, -6.02966528]]),
 array([[ 0.        ,  0.        ],
        [-3.59845072, -4.29039113],
        [-5.77987454, -7.4544766 ],
        [-6.53365453, -6.72981021],
        [-2.45649251, -3.81941557],
        [-3.65250094, -5.37488878]])]

ONE TEST: POLICY FORMED

Mouse

-6.10693584, -5.0698584

-2.33158313, -2.278446

0        ,  0.       

4.11305957,  3.6347789
  
4.60095968,  4.04064418

3.97011286,  3.10334809


CAT

0        , 0.       

8.58297923, 7.96147233

8.05288662, 7.98932263

3.32721638, 5.1754332

8.29258907, 9.12580202

8.10549219, 9.80908741

#edited Training loop (to fix?)

In [112]:
class CardGameEnvEdited(py_environment.PyEnvironment):

  number_steps_index = 0
  mouse_index = 1
  cat_index = 2
  move_reward = 1
  end_punishment = -10
  stop_punishment = 0

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(2,), dtype=np.int32, minimum=[0,0], maximum=[1,1], name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,3), dtype=np.float32, minimum=0, name='observation')

    self._state = [0, 2, 4]
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = [0, 2, 4]
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.float32))


  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # check action
    #print('Action: ', action)
    #print('State before: ', self._state)
    cost_tally = [0, 0]
    if action[0] == 0: #if mouse
      if action[1] == 1: #if close
        cost_tally[0] -= self.stop_punishment
      else:
        #move the mouse
        mouse_location = self._state[self.mouse_index]
        if mouse_location == 1:
            self._state[self.mouse_index] = 3
        elif mouse_location == 2:
            self._state[self.mouse_index] = 1
        else:
            self._state[self.mouse_index] = 2

        cost_tally[0] += self.move_reward
        if self._state[self.mouse_index] == 3: #if mouse in room 3
          cost_tally[1] += self.move_reward
    else:
      if action[1] == 1: #if close
        cost_tally[1] -= self.stop_punishment
      else:
        #move the cat
        cat_location = self._state[self.cat_index]
        #print(f"cat starting location {cat_location}")
        if cat_location == 3:
            self._state[self.cat_index] = 4
        elif cat_location == 4:
            self._state[self.cat_index] = 5
        else:
            self._state[self.cat_index] = 3
        #print(f"cat ending location {self._state[self.cat_index]}")

        cost_tally[1] += self.move_reward
        if self._state[self.cat_index] == 3: #if cat in room 3
          cost_tally[0] += self.move_reward
    #print('Action data type: ', type(action))


    # calc state
    if self._state[self.mouse_index] == self._state[self.cat_index]:
      #print(f"cat and mouse met, state {self._state}")
      self._episode_ended = True
    self._state[0] += 1
    #print('State after: ', self._state)


    if self._episode_ended:
      reward = self.end_punishment
      return ts.termination(np.array([self._state],  dtype=np.float32), reward)
    elif self._state[0] > 20:
      return ts.termination(np.array([self._state],  dtype=np.float32), 0)
    else:
      reward = cost_tally[action[0]] + np.random.normal(0, 0.1) #ADD RANDOM NOISE TO REWARD AS SPECIFIED BY PAPER
      return ts.transition(
          np.array([self._state], dtype=np.float32), reward=reward, discount=1.0)

In [113]:
def train(n_training_episodes, epsilon, alpha, gamma, env, max_steps, mouse_S, cat_S):
  for episode in range(n_training_episodes):

    # Reset the environment
    info, reward, done, state = env.reset()
    state = state[0]
    cat_state = state_conversion_cat(state)
    mouse_state = state_conversion_mouse(state)


    # repeat
    for step in range(max_steps):

      #decide if moving cat or mouse 0 = mouse
      move = random.choice([0, 1])
      if move == 0:
        action = epsilon_greedy_policy(mouse_S, mouse_state, epsilon)
      else:
        action = epsilon_greedy_policy(cat_S, cat_state, epsilon)


      info, reward, done, new_state = env.step([move, action])
      new_state = new_state[0] #CHECK THIS
      new_cat_state = state_conversion_cat(new_state)
      new_mouse_state = state_conversion_mouse(new_state)

      #print(f"starting state {state}, new_state {new_state}")
      #print(f"move made: {move} {action}")


      #update policy (need to go through equations to make sure it's accurate)
      if move == 0:
        update_table(mouse_S, mouse_state, action, alpha, reward, gamma, new_mouse_state)  #update mouse
      else:
        update_table(cat_S, cat_state, action, alpha, reward, gamma, new_cat_state)  #update cat

      # If done, finish the episode
      if not done:
        #print(f"finished episode, end state {new_state}")
        break

      # Our state is the new state
      state = new_state
      mouse_state = new_mouse_state
      cat_state = new_cat_state

  return [mouse_S, cat_S]

In [114]:
#Hyperparameters
alpha = 0.1
gamma = 0.9
epsilon = 0.1
n_training_episodes = 10000
learning_rate = 0.1  #alpha

# Evaluation parameters
n_eval_episodes = 100

#environment
env = CardGameEnvEdited()

#Q-tables
state_space = 6
action_space = 2
mouse_Qtable = initialize_q_table(state_space, action_space)
cat_Qtable = initialize_q_table(state_space, action_space)

/usr/local/lib/python3.10/dist-packages/tf_agents/specs/array_spec.py:352: RuntimeWarning: invalid value encountered in cast
  self._minimum[self._minimum == -np.inf] = low
/usr/local/lib/python3.10/dist-packages/tf_agents/specs/array_spec.py:353: RuntimeWarning: invalid value encountered in cast
  self._minimum[self._minimum == np.inf] = high


In [115]:
def convert(arr):
  out = []
  for i in range(len(arr)):
    if arr[i][0] > arr[i][1]:
      out.append("move")
    else:
      out.append("stop")
  return out

In [116]:
def print_clear(m, c):
  mm = ["13", "23", "33", "1?", "2?", "3?"]
  cc = ["33", "34", "35", "?3", "?4", "?5"]
  print("MOUSE")
  for i in range(len(m)):
    print(f"{mm[i]} {m[i]}")

  print("CAT")
  for i in range(len(m)):
    print(f"{cc[i]} {c[i]}")

In [117]:
mouse, cat = train(n_training_episodes, epsilon , alpha, gamma, env, 20,  mouse_Qtable, cat_Qtable)
print(mouse)
print(cat)

[[-10.           0.10155894]
 [  1.10168522   0.91609397]
 [  0.           0.        ]
 [  9.96877845   9.01483316]
 [  9.97530568   8.97543512]
 [  9.97069427   8.99649132]]
[[  0.           0.        ]
 [  1.04882979   0.87487941]
 [-10.           0.01381633]
 [ 10.05632259   9.06499858]
 [ 10.03458288   9.01096959]
 [ 10.09192254   9.03855527]]


In [118]:
print_clear(convert(mouse), convert(cat))

MOUSE
13 stop
23 move
33 stop
1? move
2? move
3? move
CAT
33 stop
34 move
35 stop
?3 move
?4 move
?5 move


In [119]:
def cat_junk(start_state, end_state):
  cat_state = state_conversion_cat(start_state)
  new_cat_state = state_conversion_cat(end_state)
  update_table(cat_q, cat_state, 0, alpha, reward, gamma, new_cat_state)



In [120]:
def mouse_junk(start_state, end_state):
  mouse_state = state_conversion_mouse(start_state)
  new_mouse_state = state_conversion_mouse(end_state)
  update_table(mouse_q, mouse_state, 0, alpha, reward, gamma, new_mouse_state)

In [121]:
mouse_q = initialize_q_table(6, 2)
cat_q = initialize_q_table(6, 2)
info, reward, done, state = env.reset()
info, reward, done, state2 = env.step([1,0])
print(reward)
cat_junk(state[0], state2[0])
print(cat_q)
info, reward, done, state3 = env.step([1,0])
cat_junk(state2[0], state3[0])
print(cat_q)
print(reward)
info, reward, done, state4 = env.step([0,0])
mouse_junk(state3[0], state4[0])
print(mouse_q)
info, reward, done, state5 = env.step([0,0])
mouse_junk(state4[0], state5[0])
print(state5)
print(mouse_q)
print(reward)
print(done)
info, reward, done, state6 = env.step([0,0])
print(done)
print(reward)
print(state6)

0.8005183
[[0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.08005183 0.        ]
 [0.         0.        ]]
[[0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.08005183 0.        ]
 [0.10751667 0.        ]]
1.0751667
[[0.         0.        ]
 [0.10726993 0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]
[[4. 3. 3.]]
[[-1.          0.        ]
 [ 0.10726993  0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]]
-10.0
0.0
1.0
0.0
[[0. 2. 4.]]
